### Import libraries and functions

In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import arviz as az
import pymc as pm
from datetime import datetime, timedelta

def generate_sample_data():
    # Generate sample data
    np.random.seed(42)  # For reproducibility
    # Generate dates
    start_date = datetime(2000, 1, 1)
    end_date = datetime(2023, 12, 31)
    dates = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]
    # Generate ISINs and CUSIPs
    isins = ['ISIN001', 'ISIN002', 'ISIN003']
    cusips = ['CUSIP001', 'CUSIP002', 'CUSIP003']
    # Generate random prices, modified duration, and Z-spread
    data = []
    for date in dates:
        for isin in isins:
            price = np.random.uniform(90, 110)
            moddur_m = np.random.uniform(1, 10)
            zsprd_m = np.random.uniform(1, 1500)
            data.append([date.strftime('%Y%m%d'), isin, None, price, moddur_m, zsprd_m])
        for cusip in cusips:
            price = np.random.uniform(90, 110)
            moddur_m = np.random.uniform(1, 10)
            zsprd_m = np.random.uniform(1, 1500)
            data.append([date.strftime('%Y%m%d'), None, cusip, price, moddur_m, zsprd_m])
    # Create DataFrame
    columns = ['Date', 'ISIN', 'CUSIP', 'PRICE', 'MODDUR_M', 'ZSPRD_M']
    df = pd.DataFrame(data, columns=columns)
    return df

def dataset(df, identifier, mpr):
    """
    Preprocess the dataset based on the identifier (ISIN or CUSIP) and calculate the return (RT).
    """
    if identifier == 'ISIN':
        df = df[['Date', 'ISIN', 'PRICE', 'MODDUR_M', 'ZSPRD_M']].dropna()
        df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d').dt.date
        df.set_index(['Date', 'ISIN'], inplace=True)
        df['RT'] = df.groupby('ISIN')['PRICE'].pct_change(mpr)
        df = df.dropna()
    elif identifier == 'CUSIP':
        df = df[['Date', 'CUSIP', 'PRICE', 'MODDUR_M', 'ZSPRD_M']].dropna()
        df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d').dt.date
        df.set_index(['Date', 'CUSIP'], inplace=True)
        df['RT'] = df.groupby('CUSIP')['PRICE'].pct_change(mpr)
        df = df.dropna()
    return df

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


### Load for Generate Sample Data

In [3]:
# Generate sample data
data = generate_sample_data()

# Preprocess data for CUSIPs with a period of 5
data_set = dataset(data, "CUSIP", 5)

# Extract the relevant columns
X = data_set[['MODDUR_M', 'ZSPRD_M']].values
y = data_set['RT'].values

# Split into training and out-of-sample data
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

### Define the Bayesian regression Model

In [6]:
with pm.Model() as bayesian_regression:
    # Priors for the regression coefficients
    alpha = pm.Normal('alpha', mu=0, sigma=10)  # Intercept
    beta = pm.Normal('beta', mu=0, sigma=10, shape=2)  # Coefficients for MODDUR_M and ZSPRD_M
    
    # Standard deviation of the error term
    sigma = pm.HalfNormal('sigma', sigma=10)
    
    # Linear model
    mu = alpha + pm.math.dot(X_train, beta)
    
    # Likelihood (99% quantile regression)
    # We use a custom likelihood to model the 99th percentile
    tau = 0.99  # 99th percentile
    likelihood = pm.StudentT('likelihood', nu=3, mu=mu, sigma=sigma, observed=y_train)

### Sample from the Posterior

In [10]:
with bayesian_regression:
    # Sample from the posterior
    trace = pm.sample(2000, tune=1000, chains=2, cores=2)

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: ExpandDims{axis=0}(10)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/p311-env/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1920, in process_node
    e, self, [(x, None) for x in node.outputs], node_rewriter, node
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/p311-env/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1081, in transform
    if not (
       ^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/p311-env/lib/python3.11/site-packages/pytensor/tensor/rewriting/basic.py", line 1121, in constant_folding
  File "/opt/homebrew/Caskroom/miniconda/base/envs/p311-env/lib/python3.11/site-packages/pytensor/link/c/op.py", line 119,


You can find the C code in this temporary file: /var/folders/k6/94glr3fd7xz9yfzkq_2g0tgw0000gn/T/pytensor_compilation_error_h7p4qz6e


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: ExpandDims{axis=0}(0)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/p311-env/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1920, in process_node
    e, self, [(x, None) for x in node.outputs], node_rewriter, node
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/p311-env/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1081, in transform
    if not (
       ^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/p311-env/lib/python3.11/site-packages/pytensor/tensor/rewriting/basic.py", line 1121, in constant_folding
  File "/opt/homebrew/Caskroom/miniconda/base/envs/p311-env/lib/python3.11/site-packages/pytensor/link/c/op.py", line 119, 


You can find the C code in this temporary file: /var/folders/k6/94glr3fd7xz9yfzkq_2g0tgw0000gn/T/pytensor_compilation_error_zb2fek_e


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: ExpandDims{axis=0}(3)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/p311-env/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1920, in process_node
    e, self, [(x, None) for x in node.outputs], node_rewriter, node
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/p311-env/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1081, in transform
    if not (
       ^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/p311-env/lib/python3.11/site-packages/pytensor/tensor/rewriting/basic.py", line 1121, in constant_folding
  File "/opt/homebrew/Caskroom/miniconda/base/envs/p311-env/lib/python3.11/site-packages/pytensor/link/c/op.py", line 119, 

...

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: ExpandDims{axis=0}(0)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/p311-env/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1920, in process_node
    e, self, [(x, None) for x in node.outputs], node_rewriter, node
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/p311-env/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1081, in transform
    if not (
       ^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/p311-env/lib/python3.11/site-packages/pytensor/tensor/rewriting/basic.py", line 1121, in constant_folding
  File "/opt/homebrew/Caskroom/miniconda/base/envs/p311-env/lib/python3.11/site-packages/pytensor/link/c/op.py", line 119, 


You can find the C code in this temporary file: /var/folders/k6/94glr3fd7xz9yfzkq_2g0tgw0000gn/T/pytensor_compilation_error_t98it0na


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: Cast{float64}(10)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/p311-env/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1920, in process_node
    e, self, [(x, None) for x in node.outputs], node_rewriter, node
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/p311-env/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1081, in transform
    if not (
       ^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/p311-env/lib/python3.11/site-packages/pytensor/tensor/rewriting/basic.py", line 1121, in constant_folding
  File "/opt/homebrew/Caskroom/miniconda/base/envs/p311-env/lib/python3.11/site-packages/pytensor/link/c/op.py", line 119, in m


You can find the C code in this temporary file: /var/folders/k6/94glr3fd7xz9yfzkq_2g0tgw0000gn/T/pytensor_compilation_error_vf1f_vdu


TypeError: unsupported operand type(s) for /: 'str' and 'str'